In [1]:
import scanpy as sc
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rcParams
from matplotlib import colors
from matplotlib import patches
import seaborn as sns
import batchglm
import diffxpy.api as de
import patsy as pat
from statsmodels.stats.multitest import multipletests
import logging, warnings
import statsmodels.api as sm

In [2]:
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()
de.__version__

logging.getLogger("tensorflow").setLevel(logging.ERROR)
logging.getLogger("batchglm").setLevel(logging.INFO)
logging.getLogger("diffxpy").setLevel(logging.INFO)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 35)
warnings.filterwarnings("ignore", category=DeprecationWarning, module="tensorflow")

-----
anndata     0.8.0
scanpy      1.8.2
sinfo       0.3.1
-----
PIL                 8.0.1
anndata             0.8.0
asttokens           NA
backcall            0.2.0
batchglm            v0.7.4
cairo               1.20.0
cffi                1.14.3
cloudpickle         1.6.0
colorama            0.4.4
comm                0.1.2
cycler              0.10.0
cython_runtime      NA
cytoolz             0.11.0
dask                2.30.0
dateutil            2.8.2
debugpy             1.6.6
decorator           4.4.2
diffxpy             v0.7.4
executing           1.2.0
h5py                3.1.0
igraph              0.9.1
ipykernel           6.21.3
jedi                0.17.2
joblib              1.1.0
kiwisolver          1.3.1
leidenalg           0.8.3
llvmlite            0.34.0
louvain             0.6.1
matplotlib          3.5.1
matplotlib_inline   0.1.6
mpl_toolkits        NA
natsort             7.1.0
numba               0.51.2
numexpr             2.7.1
numpy               1.22.1
packaging           2

In [3]:
sc.settings.n_jobs = 40
sc.set_figure_params(figsize=(4, 4), vector_friendly = True)
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

In [4]:
adata = sc.read_h5ad("Output_230911_adata_scvi_random_sampleID_annot.h5ad") # 6-10m

In [5]:
adata

AnnData object with n_obs × n_vars = 393060 × 49133
    obs: 'batch', 'sampleID', 'Age', 'Assay', 'Stage', 'Race', 'PMI', 'Hemisphere', 'Library', 'Brain_Region', 'Dataset', 'Sex', 'Diagnosis', 'DF_classification', 'cluster_original', 'cluster_main', 'n_genes', 'Stage2', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'doublet_scores', 'predicted_doublets', '_scvi_batch', '_scvi_labels', 'leiden_scvi', 'leiden_0.5', 'leiden_0.7', 'leiden_1.0', 'cluster_main2', 'Brain_Region2', 'Brain_Region3', 'Brain_Region_Unit', 'cluster_number'
    uns: 'Brain_Region3_colors', 'Stage2_colors', 'cluster_main2_colors', 'leiden_scvi_colors', 'log1p'
    obsm: 'X_scVI_sampleID', 'X_umap', '_scvi_extra_categorical_covs'
    layers: 'counts', 'logcounts', 'scaled'

In [6]:
adata.obs["Stage"].value_counts()

Adult          187605
Fetal          109726
Childhood       29214
Infancy         26651
Adolescence     24256
Neonatal        15608
Name: Stage, dtype: int64

In [7]:
adata.obs["Stage3"] =  adata.obs['Stage2'].astype(str)

adata.obs.loc[(adata.obs['Stage2'] == 'Fetal (1st trimester)'), 'Stage3'] = 'Fetal'
adata.obs.loc[(adata.obs['Stage2'] == 'Fetal (2nd trimester)'), 'Stage3'] = 'Fetal'
adata.obs.loc[(adata.obs['Stage2'] == 'Fetal (3rd trimester)'), 'Stage3'] = 'Fetal'

adata.obs.loc[(adata.obs['Stage2'] == 'Childhood (1-6Y)'), 'Stage3'] = 'Childhood'
adata.obs.loc[(adata.obs['Stage2'] == 'Childhood (6-12Y)'), 'Stage3'] = 'Childhood'

adata.obs.loc[(adata.obs['Stage2'] == 'Adolescence (12-20Y)'), 'Stage3'] = 'Adolescence'

adata.obs.loc[(adata.obs['Stage2'] == 'Adult (20-40Y)'), 'Stage3'] = 'Adult'
adata.obs.loc[(adata.obs['Stage2'] == 'Adult (40-60Y)'), 'Stage3'] = 'Adult'
adata.obs.loc[(adata.obs['Stage2'] == 'Adult (60-80Y)'), 'Stage3'] = 'Adult'
adata.obs.loc[(adata.obs['Stage2'] == 'Adult (>80Y)'), 'Stage3'] = 'Adult'

adata.obs.Stage3.value_counts()

Adult          187605
Fetal          109726
Childhood       44465
Neonatal        37254
Adolescence     14010
Name: Stage3, dtype: int64

In [8]:
adata1 = adata[adata.obs["cluster_number"] == "C11"].copy()
adata2 = adata[(adata.obs["cluster_number"] == "C3") | (adata.obs["cluster_number"] == "C14") | (adata.obs["cluster_number"] == "C27")].copy()

In [9]:
adata1.obs['total_counts_scaled'] = adata1.obs['total_counts']/adata1.obs['total_counts'].mean()
adata2.obs['total_counts_scaled'] = adata2.obs['total_counts']/adata2.obs['total_counts'].mean()

# Fetal Sex

In [10]:
# adata = adata[adata.obs["Stage3"] == "Fetal"].copy()

In [11]:
# counts = adata.obs["sampleID"].value_counts()
# sorted_counts = counts.sort_index()
# sorted_counts

In [12]:
# plt.rcParams['figure.figsize']=(10,3)
# sc.pl.violin(adata, ['XIST'], groupby = "sampleID", rotation=90)

In [13]:
import pandas as pd

# Create a list of female SampleIDs
female_sample_ids = ["10X116_6", "10X119_2", "10X119_5", "10X156_2", "10X167_8", "10X168_1", "10X168_2", "10X168_5", "10X169_1", "10X169_2", "10X177_4", "10X178_5",
                     "10X212_5", "10X212_6", "10X213_1", "10X302_1", "510_PFC_B1", "510_WGE_B1", "611_PFC_B1", "611_WGE_B1", "993_PFC_B2", "993_WGE_B2", "RL2121"]  # Add all female SampleIDs

# Assuming adata is your AnnData object
# Create an initial 'Predicted_Sex' column with a default label (e.g., "Male")
adata2.obs["Predicted_Sex"] = "Male"

# Apply the condition to assign sex labels
adata2.obs.loc[adata2.obs["sampleID"].isin(female_sample_ids), "Predicted_Sex"] = "Female"

adata2.obs[["Sex", "Predicted_Sex", "Stage2"]].value_counts()

Sex      Predicted_Sex  Stage2               
Unknown  Male           Fetal (1st trimester)    21958
         Female         Fetal (1st trimester)     9008
F        Female         Fetal (2nd trimester)     3611
M        Male           Fetal (2nd trimester)      218
                        Adult (20-40Y)              69
F        Male           Neonatal                    16
                        Adult (20-40Y)              14
         Female         Fetal (3rd trimester)       14
M        Male           Childhood (1-6Y)            13
                        Adult (60-80Y)              12
F        Male           Adult (40-60Y)              12
                        Adolescence (12-20Y)        12
M        Male           Neonatal                     9
                        Childhood (6-12Y)            5
F        Male           Adult (>80Y)                 3
                        Adult (60-80Y)               3
M        Male           Adult (>80Y)                 2
                   

## ESR1

In [14]:
formula = "1 + Sex + Stage3(reference='Fetal') + Brain_Region(reference='prefrontal cortex') + n_genes_by_counts + pct_counts_mt"

In [15]:
adata1.obs["Stage3"].value_counts()

Adult          7866
Neonatal       2334
Childhood      1641
Fetal           774
Adolescence     281
Name: Stage3, dtype: int64

In [16]:
adata1.obs["Dataset"].value_counts()

Herring     5372
AllenM1     4175
Turecki     1632
ZhangPD     1123
Morabito     311
Hardwick     280
Braun          3
Name: Dataset, dtype: int64

In [17]:
adata1.obs["Sex"].value_counts()

M          7907
F          4986
Unknown       3
Name: Sex, dtype: int64

In [18]:
adata1.obs["Brain_Region"].value_counts()

BA9                  7106
M1                   4175
BA8                   667
BA46                  415
prefrontal cortex     311
BA10                  219
Brain                   3
Name: Brain_Region, dtype: int64

In [19]:
adata1 = adata1[(adata1.obs["Brain_Region"] != "Brain") & (adata1.obs["Brain_Region"] != "prefrontal cortex")]

In [20]:
dmat1 = de.utils.design_matrix(
    data=adata1,
    formula="~" + formula,
    as_numeric=["n_genes_by_counts", "pct_counts_mt"],
    return_type="patsy"
)

In [21]:
if np.linalg.matrix_rank(np.asarray(dmat1[0])) < np.min(dmat1[0].shape):
        print(f'Cannot test as design matrix is not full rank.')

In [22]:
np.linalg.matrix_rank(np.asarray(dmat1[0]))

12

In [23]:
np.min(dmat1[0].shape)

12

In [24]:
adata1 = adata1[:, 'ESR1']

# ESR2

In [65]:
formula = "1 + Predicted_Sex + Age_convert + Brain_Region_Unit + n_genes_by_counts + pct_counts_mt"

In [66]:
adata2 = adata2[adata2.obs["Stage"] == "Fetal"]

In [67]:
adata2.obs["Dataset"].value_counts()

Braun      27862
Cameron     3611
Herring      232
Name: Dataset, dtype: int64

In [68]:
adata2.obs[["Age", "Dataset"]].value_counts()

Age    Dataset
6.9    Braun      8147
101.5  Cameron    3611
8.0    Braun      3428
6.7    Braun      2944
12.0   Braun      2810
10.0   Braun      2444
5.0    Braun      2096
6.6    Braun      1945
13.0   Braun      1369
9.2    Braun      1112
14.0   Braun      1077
8.5    Braun       308
7.0    Braun       182
154.0  Herring     123
168.0  Herring      95
238.0  Herring      14
dtype: int64

In [69]:
import pandas as pd

# Assuming adata is your AnnData object
# Create an initial 'Predicted_Sex' column with a default label (e.g., "Male")
adata2.obs["Age_convert"] = adata2.obs["Age"].copy()

/tmp/ipykernel_1374071/2322673627.py:5: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata2.obs["Age_convert"] = adata2.obs["Age"].copy()


In [70]:
# Apply the condition to assign sex labels
adata2.obs.loc[adata2.obs["Dataset"] == "Braun", "Age_convert"] = adata2.obs.loc[adata2.obs["Dataset"] == "Braun", "Age"].apply(lambda age: (int(age) + 2) * 7)

# Display Age and Age_convert columns
print(adata2.obs[["Age", "Age_convert"]].value_counts())

Age    Age_convert
6.9    56.0           8147
101.5  101.5          3611
8.0    70.0           3428
6.7    56.0           2944
12.0   98.0           2810
10.0   84.0           2444
5.0    49.0           2096
6.6    56.0           1945
13.0   105.0          1369
9.2    77.0           1112
14.0   112.0          1077
8.5    70.0            308
7.0    63.0            182
154.0  154.0           123
168.0  168.0            95
238.0  238.0            14
dtype: int64


In [71]:
adata2.obs["sampleID"].value_counts()

10X89_2       1716
10X152_2      1332
10X177_4      1217
10X199_8      1093
10X198_3      1070
10X101_3      1028
10X185_8       891
10X178_5       879
993_WGE_B2     872
10X119_5       836
10X152_5       829
10X99_5        815
10X199_2       787
10X152_7       783
10X102_1       765
10X99_6        757
10X119_2       716
10X185_6       653
10X101_7       651
10X168_2       644
10X213_1       641
611_WGE_B1     607
10X122_1       604
10X169_1       587
10X124_3       565
510_PFC_B1     560
993_PFC_B2     555
10X168_1       553
10X167_8       551
510_WGE_B1     546
10X169_2       515
10X187_5       514
10X168_5       491
611_PFC_B1     471
10X115_5       451
10X188_2       403
10X255_2       401
10X110_4       393
10X212_5       390
10X124_5       374
10X187_2       349
10X212_6       338
10X115_7       333
10X116_6       328
10X125_2       313
10X254_4       311
10X163_4       308
10X252_5       218
10X302_1       182
10X252_3       147
10X156_2       140
RL2103         123
RL2107      

In [72]:
adata2.obs["Predicted_Sex"].value_counts()

Male      19072
Female    12633
Name: Predicted_Sex, dtype: int64

In [73]:
adata2.obs["Brain_Region_Unit"].value_counts()

Cerebral cortex        7372
Thalamus               5078
Cerebellum             3985
Forebrain              3518
Midbrain               3343
Medulla                2253
Ganglionic Eminence    2025
Pons                   1164
Hypothalamus           1070
Hindbrain               835
Hippocampus             661
Striatum                401
Name: Brain_Region_Unit, dtype: int64

In [74]:
adata2 = adata2[adata2.obs["Brain_Region_Unit"] != "Uncategorized"]

In [75]:
dmat2 = de.utils.design_matrix(
    data=adata2,
    formula="~" + formula,
    as_numeric=["Age_convert", "n_genes_by_counts", "pct_counts_mt"],
    return_type="patsy"
)

In [76]:
if np.linalg.matrix_rank(np.asarray(dmat2[0])) < np.min(dmat2[0].shape):
        print(f'Cannot test as design matrix is not full rank.')

In [77]:
adata2 = adata2[:, 'ESR2']

# GLM model

In [110]:
for i, gene in enumerate(adata1.var_names):
    # Specify model
    pois_model = sm.GLM(
        endog=adata1.X[:, i].todense(), #[idx_train, :], 
        exog=dmat1[0], 
        offset=np.log(adata1.obs['total_counts_scaled'].values),
        family=sm.families.Poisson(),
        
    )

    # Fit the model
    pois_results1 = pois_model.fit() # i enumerate 하지 않으면 error가 발생.

    # Get the covariance matrix
    cov_mat1 = pois_results1.cov_params()

In [111]:
p_val = pois_results1.pvalues
p_val

array([5.76092690e-007, 2.02425058e-002, 3.42365261e-002, 2.21229220e-005,
       1.82171745e-013, 2.40695049e-007, 3.92038436e-015, 8.10610249e-005,
       3.20724819e-001, 5.14164670e-097, 2.30223666e-115, 8.01752419e-015])

In [112]:
formatted_p_values = [format(p, '.4f') for p in p_val]
formatted_p_values

['0.0000',
 '0.0202',
 '0.0342',
 '0.0000',
 '0.0000',
 '0.0000',
 '0.0000',
 '0.0001',
 '0.3207',
 '0.0000',
 '0.0000',
 '0.0000']

In [113]:
# alpha = 0.05  # Set your desired alpha level
# reject, corrected_fdr, _, _ = multipletests(p_val, alpha=alpha, method='fdr_bh')
# formatted_fdr = [format(p, '.4f') for p in corrected_fdr]
# formatted_fdr

In [114]:
summary_with_names1 = pois_results1.summary(xname=dmat1[1])
summary_with_names1

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                12582
Model:                            GLM   Df Residuals:                    12570
Model Family:                 Poisson   Df Model:                           11
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -4667.9
Date:                Tue, 12 Sep 2023   Deviance:                       4937.0
Time:                        16:41:11   Pearson chi2:                 8.51e+03
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.6252      0.125      4.999      0.000       0.380       0.870
Sex[T.M]                 0.1633      0.070      2.322      0.020       0.025       0.301
Stage3[T.Adult]         -0.2364      0.112     -2.117      0.034      -0.455      -0.018
Stage3[T.Childhood]      0.4943      0.117      4.242      0.000       0.266       0.723
Stage3[T.Fetal]         -2.6373      0.358     -7.361      0.000      -3.339      -1.935
Stage3[T.Neonatal]      -0.6607      0.128     -5.165      0.000      -0.911      -0.410
Brain_Region[T.BA9]     -0.5949      0.076     -7.857      0.000      -0.743      -0.446
Brain_Region[T.BA10]     0.4633      0.118      3.941      0.000       0.233       0.694
Brain_Region[T.BA46]     0.0889      0.089      0.993      0.321      -0.087       0.264
Brain_Region[T.M1]      -2.5529      0.122    -20.902      0.000      -2.792      -2.314
n_genes_by_counts       -0.0004   1.59e-05    -22.830      0.000      -0.000      -0.000
pct_counts_mt            0.0702      0.009      7.767      0.000       0.052       0.088
========================================================================================
"""

In [118]:
res = (summary_with_names1.tables[1])
res = pd.DataFrame(res[1:], columns=res[0])
res["pvalue"] = p_val
res

,,coef,std err,z,P>|z|,[0.025,0.975],pvalue
0,Intercept,0.6252,0.125,4.999,0.000,0.380,0.870,5.760927e-07
1,Sex[T.M],0.1633,0.070,2.322,0.020,0.025,0.301,2.024251e-02
2,Stage3[T.Adult],-0.2364,0.112,-2.117,0.034,-0.455,-0.018,3.423653e-02
3,Stage3[T.Childhood],0.4943,0.117,4.242,0.000,0.266,0.723,2.212292e-05
4,Stage3[T.Fetal],-2.6373,0.358,-7.361,0.000,-3.339,-1.935,1.821717e-13
5,Stage3[T.Neonatal],-0.6607,0.128,-5.165,0.000,-0.911,-0.410,2.406950e-07
6,Brain_Region[T.BA9],-0.5949,0.076,-7.857,0.000,-0.743,-0.446,3.920384e-15
7,Brain_Region[T.BA10],0.4633,0.118,3.941,0.000,0.233,0.694,8.106102e-05
8,Brain_Region[T.BA46],0.0889,0.089,0.993,0.321,-0.087,0.264,3.207248e-01
9,Brain_Region[T.M1],-2.5529,0.122,-20.902,0.000,-2.792,-2.314,5.141647e-97


In [119]:
res.to_csv("Summary_ESR1_GLM_0912.csv")

In [121]:
for i, gene in enumerate(adata2.var_names):
    # Specify model
    pois_model = sm.GLM(
        endog=adata2.X[:, i].todense(), #[idx_train, :], 
        exog=dmat2[0], 
        offset=np.log(adata2.obs['total_counts_scaled'].values),
        family=sm.families.Poisson()
    )

    # Fit the model
    pois_results2 = pois_model.fit() # i enumerate 하지 않으면 error가 발생.

    # Get the covariance matrix
    cov_mat2 = pois_results2.cov_params()

In [122]:
summary_with_names2 = pois_results2.summary(xname=dmat2[1])

In [123]:
p_val = pois_results2.pvalues
p_val

array([4.85216239e-004, 1.05211702e-002, 6.42332927e-100, 9.11199383e-026,
       2.40821673e-039, 1.38081505e-003, 1.41776124e-028, 2.78128070e-002,
       1.36049489e-007, 7.83317128e-011, 1.19167917e-005, 3.00838708e-005,
       2.41537958e-002, 3.26755784e-010, 0.00000000e+000, 3.12970559e-044])

In [124]:
formatted_p_values = [format(p, '.4f') for p in p_val]
formatted_p_values

['0.0005',
 '0.0105',
 '0.0000',
 '0.0000',
 '0.0000',
 '0.0014',
 '0.0000',
 '0.0278',
 '0.0000',
 '0.0000',
 '0.0000',
 '0.0000',
 '0.0242',
 '0.0000',
 '0.0000',
 '0.0000']

In [91]:
# alpha = 0.05  # Set your desired alpha level
# reject, corrected_fdr, _, _ = multipletests(p_val, alpha=alpha, method='fdr_bh')
# corrected_fdr
# formatted_fdr = [format(p, '.4f') for p in corrected_fdr]
# formatted_fdr

array([6.46954985e-04, 1.20241946e-02, 5.13866341e-99, 2.42986502e-25,
       9.63286691e-39, 1.69946468e-03, 4.53683596e-28, 2.78128070e-02,
       2.41865758e-07, 1.79043915e-10, 1.90668667e-05, 4.37583576e-05,
       2.57640488e-02, 6.53511569e-10, 0.00000000e+00, 1.66917631e-43])

In [94]:
print(summary_with_names2)

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                31705
Model:                            GLM   Df Residuals:                    31689
Model Family:                 Poisson   Df Model:                           15
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -18037.
Date:                Tue, 12 Sep 2023   Deviance:                       18276.
Time:                        16:28:03   Pearson chi2:                 2.43e+04
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

In [125]:
res = (summary_with_names2.tables[1])
res = res = pd.DataFrame(res[1:], columns=res[0])
res["pvalue"] = p_val
res

,,coef,std err,z,P>|z|,[0.025,0.975],pvalue
0,Intercept,-0.2520,0.072,-3.489,0.000,-0.394,-0.110,4.852162e-04
1,Predicted_Sex[T.Male],0.0716,0.028,2.558,0.011,0.017,0.127,1.052117e-02
2,Brain_Region_Unit[T.Cerebral cortex],0.9683,0.046,21.219,0.000,0.879,1.058,6.423329e-100
3,Brain_Region_Unit[T.Forebrain],0.5527,0.053,10.495,0.000,0.449,0.656,9.111994e-26
4,Brain_Region_Unit[T.Ganglionic Eminence],0.8028,0.061,13.124,0.000,0.683,0.923,2.408217e-39
5,Brain_Region_Unit[T.Hindbrain],-0.3847,0.120,-3.199,0.001,-0.620,-0.149,1.380815e-03
6,Brain_Region_Unit[T.Hippocampus],0.8982,0.081,11.089,0.000,0.739,1.057,1.417761e-28
7,Brain_Region_Unit[T.Hypothalamus],-0.2162,0.098,-2.200,0.028,-0.409,-0.024,2.781281e-02
8,Brain_Region_Unit[T.Medulla],-0.4057,0.077,-5.271,0.000,-0.557,-0.255,1.360495e-07
9,Brain_Region_Unit[T.Midbrain],-0.4903,0.075,-6.504,0.000,-0.638,-0.343,7.833171e-11


In [126]:
res.to_csv("Summary_ESR2_GLM_0912.csv")